* Lägg till oändligheten i datamängden, dvs se till att alla edge-cases är hanterade.
* Till exempel: Turbulensmodell ska kunna hantera när flödet är laminärt.



## Fixing robustness issues

We first introduce the simulation quantities.

#### Background: Simulation on pipe, hyper parameters
        
* Macro Problem  
    * `eps: float` size and wavelength of roughness
    * `xDim, yDim: int[odd]` , number of Fourier/Chebyshev Basis functions. Odd integer.
* Micro Problem
    * `nMic: int`, number of micro problems
    * `xPos: list[float]`, positions of micro domains. Uniform start at x=-1.
    * `width: float = eps * 5`, width of micro domain.0
    * `height: float = width * 0.8`, height of micro domain.
    * `corner_w: float = 0.2*width`, radius of corner rounding
    * `n_refine: int`, number of dyadic refinements of mesh.
* Micro <- Macro coupling
    * `deg_project: int = 8` , degree of polynomial for extrapolation. (8 is standard)
    * `xDim_reduce, yDim_reduce: int = 5` , number of Fourier/Chebyshev modes

### Data:

We (now) have two main datasets:

DATA 1: High variance
* ``RoundedMicroGeomV2``: Uses balls to smooth corners
* ``width``: $U[0.2,1.2]$, Width
* ``scale``: $10^{-U[0.4, 1.4]}$, scale of Gaussian Process
* ``height``: $U[0.7, 1.2]$, Height of problem
* ``corner_w``: $U[0.1, 0.3]$,  Width of rounding corner
* ``line_pos``: $U[0,0.1]$, Vertical line position

DATA 2: Low variance (changes marked in <span style="color:red"> red </span>):
* ``RoundedMicroGeomV2``: Uses balls to smooth corners
* ``width``: $U[0.5,1.]$, <span style="color:red"> Width </span>
* ``scale``: $0.01 + 10^{-U[1, 2]}$,<span style="color:red">  scale of Gaussian Process </span>
* ``height``: $U[0.7, 1.2]$, Height of problem
* ``corner_w``: $U[0.1, 0.3]$,  Width of rounding corner
* ``line_pos``: $U[0.1,0.2]$, <span style="color:red"> Vertical line position</span>

DATA 3: Minimal variance (changes marked in <span style="color:red"> red </span>):
* ``RoundedMicroGeom``: Uses splines and fixed corner width
* ``width``: $1.$, Width
* ``scale``: $0.05$, scale of Gaussian Process
* ``height``: $1.$, Height of problem
* ``corner_w``: $0.3$,  Width of rounding corner
* ``line_pos``: $U[0.2,0.8]$, <span style="color:red"> Vertical line position</span>


### Data Processing

Training Data:
* 1% error if wrong discretization applied.
* ``tfm_inp`` and ``tfm_out`` have no reparameterisation, it is done in data generation.
* 


#### Metric 1
Let 
* $u_\epsilon$=true, 
* $\overline{u}$= HMM,
* $\tilde{u}$ =deep HMM,
* $u= \phi \circledast u_\epsilon$, low pass true

For all $v = u_\epsilon, \tilde{u}, u$ we measure the relative error
$
    E_1(v) = \frac{\| \overline{u} - v\|}{\|v\|},
$
where the norm is taken over the domain $\{y < \epsilon\log \epsilon\}$.

#### Metric 2
Let 
* $r$ true rietz
* $\tilde r$ deep rietz
* $\overline r$ coarse rietz

Then consider the relative error with $q = \overline{r}, \tilde r$
$
    E_2(q) = \frac{\|r - q\|}{\|r\|}
$

#### Metric 3
The test loss is the average relative error
$
    E_3(q) = \mathbb{E}[E_2(q)]
$
over the data.


### Experiments / Problems
We notice that 
* Coupling error vs rietz error $E_1(v) \approx \mathcal{O}(E_2(q))$. 
* Coupling error sensitive at $\mathcal{O}(E_2(q))> 0.1 = 10\%$. 
* The test loss is on the order $4\%$, But rietz error on the pipe is $10-20\%$.
* Previous loss $0.002$ (see ``part14_PRES_present.ipynb``). 
    * Error on Riesz data, first representor.
    * optimal features $[x, y, t_x, t_y, c]$.
    * Optimal width 3*7 and depth 8
    * Non-randomized data parameters (Data 1)
* HMM performs better with type 1 corner smoothing, although data is smoothed with type 2.
* HMM is less sensitive with respect to type 2 smoothing (see plots)

#### Here's plots coloured by line_pos
![png](/home/emastr/deep-micro-slip-model/data/figures/coupling_error.png)


#### Here's plots coloured by model
<img src="/home/emastr/deep-micro-slip-model/data/figures/coupling_error_types_2.png"> 

### Fixes
<b> BUG FIX 1 </b>: used ``RoundedMicroGeom`` for evaluation but ``RoundedMicroGeomV2`` for training. Changing to ``RoundedMicroGeomV2`` Caused a significant decrase in error.

<b> BUG FIX 2 </b>: The lower y-limit of the macro domain is ``linePos``, but the code sets it to $0$. This results in a bias if ``linePos``$>0$. Fixed this manually in ``part30_HMM_ml_channel_flow.ipynb``.




